In [4]:

from roboflow import Roboflow
rf = Roboflow(api_key="Q5qZgvS8HpWWVRDEzQ2s")
project = rf.workspace("mranmay-shetty").project("dronevehicle")
version = project.version(1)
dataset = version.download("yolov12")
                

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to DroneVehicle-1 in yolov12:: 100%|██████████| 35986/35986 [00:16<00:00, 2192.93it/s]


In [7]:
import os
import shutil
import random
import math

# 原始資料夾
image_dir = r"E:\論文\期刊\code\YOLO\DroneVehicle-1\all\images"
label_dir = r"E:\論文\期刊\code\YOLO\DroneVehicle-1\all\labels"
output_base = "DroneVehicle-1"

# 分割比例
train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1

assert math.isclose(train_ratio + val_ratio + test_ratio, 1.0), "分割比例總和必須為 1.0"

# 收集所有圖片檔案
image_files = [f for f in os.listdir(image_dir) if f.lower().endswith((".jpg", ".jpeg", ".png"))]
random.shuffle(image_files)

# 分配檔案
n_total = len(image_files)
n_train = int(n_total * train_ratio)
n_val = int(n_total * val_ratio)

train_files = image_files[:n_train]
val_files = image_files[n_train:n_train + n_val]
test_files = image_files[n_train + n_val:]

# 建立新資料夾
for split in ["train", "val", "test"]:
    os.makedirs(os.path.join(output_base, split, "images"), exist_ok=True)
    os.makedirs(os.path.join(output_base, split, "labels"), exist_ok=True)

# 複製影像與標註
def copy_pairs(file_list, split_name):
    for fname in file_list:
        name, ext = os.path.splitext(fname)
        img_src = os.path.join(image_dir, fname)
        lbl_src = os.path.join(label_dir, name + ".txt")

        img_dst = os.path.join(output_base, split_name, "images", fname)
        lbl_dst = os.path.join(output_base, split_name, "labels", name + ".txt")

        shutil.copyfile(img_src, img_dst)
        if os.path.exists(lbl_src):
            shutil.copyfile(lbl_src, lbl_dst)

copy_pairs(train_files, "train")
copy_pairs(val_files, "val")
copy_pairs(test_files, "test")

print("✅ 已成功分割為 train / val / test")
print(f"Train: {len(train_files)} 張")
print(f"Val:   {len(val_files)} 張")
print(f"Test:  {len(test_files)} 張")


✅ 已成功分割為 train / val / test
Train: 12593 張
Val:   3598 張
Test:  1799 張


In [9]:
# 匯入套件
from ultralytics import YOLO

# 從Roboflow下載的資料夾名稱
DATA_PATH = r"E:\論文\期刊\code\YOLO\DroneVehicle-1\data.yaml"

# 載入YOLOv12模型（YOLOv8 API相容）
model = YOLO('yolo12m.pt')  # 可以根據需求選擇其他預訓練模型 (例如 yolov8s, yolov8m, yolov8l)

# 開始訓練
model.train(
    data=DATA_PATH,
    epochs=50,          # 訓練次數可調整
    imgsz=640,          # 圖片大小
    batch=16,           # 根據硬體能力調整batch size
    name="satellite1_train"  # 訓練結果保存的名稱
)

# 評估模型表現
metrics = model.val()
print(metrics)
model.save("satellite1_train")  # 保存模型


New https://pypi.org/project/ultralytics/8.3.95 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.86  Python-3.10.16 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4080 SUPER, 16376MiB)
engine\trainer: task=detect, mode=train, model=yolo12m.pt, data=E:\\\code\YOLO\DroneVehicle-1\data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=satellite1_train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks

train: Scanning E:\論文\期刊\code\YOLO\DroneVehicle-1\train\labels... 12593 images, 336 backgrounds, 0 corrupt: 100%|██████████| 12593/12593 [00:20<00:00, 612.37it/s]


train: New cache created: E:\\\code\YOLO\DroneVehicle-1\train\labels.cache
WARNING  Box and segment counts should be equal, but got len(segments) = 199217, len(boxes) = 201568. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.


val: Scanning E:\論文\期刊\code\YOLO\DroneVehicle-1\val\labels... 3598 images, 110 backgrounds, 0 corrupt: 100%|██████████| 3598/3598 [00:06<00:00, 515.52it/s]


val: New cache created: E:\\\code\YOLO\DroneVehicle-1\val\labels.cache
WARNING  Box and segment counts should be equal, but got len(segments) = 55149, len(boxes) = 55818. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
Plotting labels to runs\detect\satellite1_train2\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001111, momentum=0.9) with parameter groups 123 weight(decay=0.0), 130 weight(decay=0.0005), 129 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\satellite1_train2
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       3598      55818      0.786      0.736      0.792      0.564
                   bus        617       1993      0.911      0.884      0.943      0.704
                   car       3308      47804      0.905      0.872      0.933      0.672
           freight car        730       1688      0.651      0.562      0.607      0.414
                 truck        850       2841      0.773      0.763      0.803      0.571
                   van        595       1492      0.691      0.602      0.673      0.457
Speed: 0.1ms preprocess, 2.8ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs\detect\satellite1_train2
Ultralytics 8.3.86  Python-3.10.16 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4080 SUPER, 16376MiB)
YOLOv12m summary (fused): 169 layers, 20,108,767 parameters, 0 gradients, 67.1 GFLOPs


val: Scanning E:\論文\期刊\code\YOLO\DroneVehicle-1\val\labels.cache... 3598 images, 110 backgrounds, 0 corrupt: 100%|██████████| 3598/3598 [00:00<?, ?it/s]

WARNING  Box and segment counts should be equal, but got len(segments) = 55149, len(boxes) = 55818. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 225/225 [00:32<00:00,  6.88it/s]


                   all       3598      55818      0.786      0.736      0.791      0.564
                   bus        617       1993      0.911      0.884      0.944      0.705
                   car       3308      47804      0.905      0.872      0.932      0.674
           freight car        730       1688      0.651      0.562      0.606      0.415
                 truck        850       2841      0.773      0.763      0.803      0.571
                   van        595       1492       0.69      0.602      0.673      0.457
Speed: 0.1ms preprocess, 4.9ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs\detect\satellite1_train22
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002028F78FCD0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B

In [14]:
model.save("satellite2_train.pt")  # 保存模型